# ABL1 complex modeling

This notebook shows the modeling of tyrosine kinase inhibitors into ABL1 using functionalities from KinoML from the commit [afe169e](https://github.com/openkinome/kinoml/commit/afe169e6eefc6fddbaaf81cf720a4e224c82696d).

In [1]:
import logging
import warnings

from openeye import oechem

# filter warnings
logging.basicConfig(level=logging.ERROR)
warnings.filterwarnings("ignore")
oechem.OEThrow.SetLevel(oechem.OEErrorLevel_Error)

from kinoml.core.ligands import Ligand
from kinoml.core.proteins import BaseProtein
from kinoml.core.systems import ProteinLigandComplex
from kinoml.features.complexes import OEKLIFSKinaseHybridDockingFeaturizer
from kinoml.utils import watermark

In [2]:
# smiles from chembl
tkis = {
    "axitinib": "CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1",
    "bosutinib": "COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC)cc23)c(Cl)cc1Cl",
    "dasatinib": "Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2CCN(CCO)CC2)n1",
    "imatinib": "Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1",
    "nilotinib": "Cc1cn(-c2cc(NC(=O)c3ccc(C)c(Nc4nccc(-c5cccnc5)n4)c3)cc(C(F)(F)F)c2)cn1",
    "ponatinib": "Cc1ccc(C(=O)Nc2ccc(CN3CCN(C)CC3)c(C(F)(F)F)c2)cc1C#Cc1cnc2cccnn12",
    "erlotinib": "C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1",
    "gefitinib": "COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1"
}

In [3]:
# initialize featurizer
featurizer = OEKLIFSKinaseHybridDockingFeaturizer(
    loop_db="~/.OpenEye/rcsb_spruce.loop_db",
    output_dir="../data/tki_complexes"
)

In [4]:
# define systems
systems = []
for name, smiles in tkis.items():
    smiles_ligand = Ligand.from_smiles(smiles=smiles, name=name)
    base_protein = BaseProtein(name="ABL1")
    base_protein.uniprot_id = "P00519"
    kinase_ligand_complex = ProteinLigandComplex(components=[base_protein, smiles_ligand])
    systems.append(kinase_ligand_complex)

In [5]:
# featurize
systems = featurizer.featurize(systems)
systems

[<ProteinLigandComplex with 2 components (<BaseProtein name=ABL1>, <OpenForceFieldLigand name=axitinib>)>,
 <ProteinLigandComplex with 2 components (<BaseProtein name=ABL1>, <OpenForceFieldLigand name=bosutinib>)>,
 <ProteinLigandComplex with 2 components (<BaseProtein name=ABL1>, <OpenForceFieldLigand name=dasatinib>)>,
 <ProteinLigandComplex with 2 components (<BaseProtein name=ABL1>, <OpenForceFieldLigand name=imatinib>)>,
 <ProteinLigandComplex with 2 components (<BaseProtein name=ABL1>, <OpenForceFieldLigand name=nilotinib>)>,
 <ProteinLigandComplex with 2 components (<BaseProtein name=ABL1>, <OpenForceFieldLigand name=ponatinib>)>,
 <ProteinLigandComplex with 2 components (<BaseProtein name=ABL1>, <OpenForceFieldLigand name=erlotinib>)>,
 <ProteinLigandComplex with 2 components (<BaseProtein name=ABL1>, <OpenForceFieldLigand name=gefitinib>)>]

All ABL1 structures in the `data/tki_complexes` directory were automatically standardized to the human wild-type ABL1 sequence according to UniProt, i.e., all missing loops built, mutated back to the wild type sequence, capped termini and renumbered according to UniProt. An aromatic nitrogen of ponatinib is wrongly protonated leading to a flipped moiety, this behaviour is a bug within the openeye toolkit and was already reported.

|tki      |kinase template|ligand template|quality|comment                                     |
|---------|---------------|---------------|-------|--------------------------------------------|
|axitinib |4wa9           |4wa9           |+++    |                                            |
|bosutinib|3ue4           |3ue4           |++     |piperazine 2+                               |
|dasatinib|2gqg           |2gqg           |+++    |                                            |
|imatinib |2hyy           |2hyy           |++     |piperazine 2+                               |
|nilotinib|3cs9           |3cs9           |+++    |                                            |
|ponatinib|3qri           |3oxz           |-      |protonated aromatic nitrogen, flipped moiety|
|erlotinib|2g1t           |4hjo           |+++    |                                            |
|gefitinib|2f4j           |2itz           |+      |chlorine moiety flipped, P-loop kinked      |

In [6]:
# print environment info
w = watermark()

Watermark
---------
Last updated: 2021-09-09T00:00:19.233591+02:00

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.27.0

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 4.19.104-microsoft-standard
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Hostname: x1

Git hash: ef164cb55c53c7a7f4c1caf2d71df479faba364a

logging: 0.5.1.2

Watermark: 2.2.0


conda info
----------
sys.version: 3.7.7 (default, May  7 2020, 21:25:33) 
...
sys.prefix: /home/david/miniconda3
sys.executable: /home/david/miniconda3/bin/python
conda location: /home/david/miniconda3/lib/python3.7/site-packages/conda
conda-build: None
conda-env: /home/david/miniconda3/bin/conda-env
user site dirs: 

CIO_TEST: <not set>
CONDA_DEFAULT_ENV: kinoml
CONDA_EXE: /home/david/miniconda3/bin/conda
CONDA_PREFIX: /home/david/miniconda3/envs/kinoml
CONDA_PREFIX_1: /home/david/miniconda3
CONDA_PROMPT_MODIFIER: (kinoml) 
CONDA_PYTHON_EXE: /home/david/miniconda3/